In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Простейшая рекуррентная сеть
В этом ноутбуке мы пройдемся по основам работы с RNN. Сегодня займемся задачей генерации текста.

In [2]:
import warnings
from typing import Iterable, Tuple
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.distributions.categorical import Categorical

warnings.filterwarnings("ignore")

В качестве обучающего датасета возьмем набор из 120 тысяч анекдотов на русском языке.
[Ссылка на данные](https://archive.org/download/120_tysyach_anekdotov) и [пост на хабре про тематическое моделирование](https://habr.com/ru/companies/otus/articles/723306/)

In [3]:
with open(r"/content/drive/MyDrive/anek_djvu.txt", "r", encoding="utf-8") as f:
    text = f.read()
text[118:500]

'|startoftext|>Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!\n\n<|startoftext|>- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...\n\n<|startoftext|>- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От со'

Мы не хотим моделировать все подряд, поэтому разобьем датасет на отдельные анекдоты.  

In [4]:
def cut_data(text):
    return text.replace("\n\n", "").split("<|startoftext|>")[1:]

In [5]:
cut_text = cut_data(text)

In [6]:
cut_text[1:6]

['Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!',
 '- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...',
 '- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.',
 'Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.',
 'Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о тебе...']

Сделаем для начала самую простую модель с токенами на уровне символов. Это значит, что каждому символу в тексте ставится в соответствие некоторое число. Некоторые способы токенизации используют части слов или, наоборот, части бинарного представления текста.

In [7]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

Напишем функции для энкодинга и декодинга нашего текста. Они будут преобразовывать список символов в список чисел и обратно.

In [8]:
def encode(sentence, vocab):
    return [vocab[sys] for sys in sentence] # List of ints

def decode(tokens, vocab):
    return "".join(vocab[toc] for toc in tokens)# list of strings

In [9]:
sentence = cut_text[3]  # Берем первую строку из подготовленного текста
encoded_sentence = encode(sentence, char2int)
decoded_sentence = decode(encoded_sentence, int2char)

print("Исходная строка:", sentence)
print("Закодированная строка:", encoded_sentence)
print("Декодированная строка:", decoded_sentence)

Исходная строка: - А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.
Закодированная строка: [105, 35, 89, 35, 192, 51, 54, 35, 161, 12, 181, 62, 26, 35, 16, 97, 161, 54, 69, 51, 70, 35, 54, 180, 35, 192, 51, 35, 161, 69, 97, 35, 116, 185, 181, 45, 26, 36, 13, 136, 105, 35, 71, 51, 69, 176, 54, 26, 97, 35, 69, 97, 35, 26, 106, 97, 59, 70, 35, 192, 185, 51, 199, 97, 70, 35, 69, 97, 54, 127, 35, 43, 54, 35, 161, 51, 190, 161, 54, 192, 97, 69, 69, 51, 112, 51, 35, 116, 185, 181, 45, 181, 35, 45, 51, 35, 12, 185, 181, 73, 69, 97, 73, 35, 106, 97, 185, 97, 35, 97, 205, 97, 35, 69, 26, 35, 185, 181, 174, 120, 35, 69, 97, 35, 45, 185, 51, 161, 180, 45, 181, 207, 161, 176, 127, 105, 35, 210, 120, 70, 35, 54, 181, 12, 35, 120, 35, 62, 97, 69, 180, 35, 161, 45, 185, 51, 161, 

Просто представления символов в виде числа не подходят для обучения моделей. На выходе должны быть вероятности всех возможных токенов из словаря. Поэтому модели удобно учить с помощью энтропии. К тому же, токены часто преобразуют из исходного представления в эмбеддинги, которые также позволяют получить более удобное представление в высокоразмерном пространстве.

В итоге векторы в модели выглядят следующим образом:
![alt_text](../additional_materials/images/char_rnn.jfif)

Задание: реализуйте метод, который преобразует батч в бинарное представление.

In [10]:
def one_hot_encode(int_words: torch.Tensor, vocab_size: int) -> torch.Tensor:
    words_one_hot = torch.zeros(
        (int_words.numel(), vocab_size), dtype=torch.float32, device=int_words.device
    )
    words_one_hot[torch.arange(words_one_hot.shape[0]), int_words.flatten().long()] = 1.0
    words_one_hot = words_one_hot.reshape((*int_words.shape, vocab_size))
    return words_one_hot


Проверьте ваш код.

In [11]:
test_seq = torch.tensor([[2, 6, 4, 1], [0,3, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

tensor([[[0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


Однако, наши последовательности на самом деле разной длины. Как же объединить их в батч?

Реализуем два необходимых класса:
- токенайзер, который будет брать текст, кодировать и декодировать символы. Еще одно, что будет реализовано там - добавлено несколько специальных символов (паддинг, конец последовательности, начало последовательности).
- Датасет, который будет брать набор шуток, используя токенайзер, строить эмбеддинги и дополнять последовательность до максимальной длины.

In [12]:
class Tokenizer:
    def __init__(self, cut_text, max_len: int = 512):
        self.text = text
        self.max_len = max_len
        self.specials = ['<pad>', '<bos>', '<eos>']
        unique_chars = tuple(set(text))
        self.int2char = dict(enumerate(tuple(set(text))))
        self.char2int = {ch: ii for ii, ch in int2char.items()}
        self._add_special("<pad>")
        self._add_special('<bos>')
        self._add_special('<eos>')

    def _add_special(self, symbol) -> None:
        # add special characters to yuor dicts
        sym_num = len(self.char2int)
        self.char2int[symbol] = sym_num
        self.int2char[sym_num] = symbol

    @property
    def vocab_size(self):
        return len(self.int2char) # your code

    def decode_symbol(self, el):
        return self.int2char[el]

    def encode_symbol(self, el):
        return self.char2int[el]

    def str_to_idx(self, chars):
        return [self.char2int[sym] for sym in chars] # str -> list[int]

    def idx_to_str(self, idx):
        return [self.int2char[toc] for toc in idx] # list[int] -> list[str]

    def encode(self, chars):
        chars = ['<bos>'] + list(chars) + ['<eos>']
        return self.str_to_idx(chars)

    def decode(self, idx):
        chars = self.idx_to_str(idx)
        return "".join(chars) # make string from list

In [13]:
class JokesDataset(Dataset):
    def __init__(self, tokenizer, cut_text, max_len: int = 512):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.cut_text = cut_text
        self.pad_index = torch.tensor(tokenizer.encode('<pad>')[0], dtype=torch.long)


    def __len__(self):
        return len(self.cut_text)

    def __getitem__(self, idx):
        text = self.cut_text[idx]
        encoded = self.tokenizer.encode(text)
        input_sequence = torch.full((self.max_len,), self.pad_index, dtype=torch.long)
        target_sequence = torch.full((self.max_len,), self.pad_index, dtype=torch.long)

        input_sequence[:min(len(encoded) - 1, self.max_len -1)] = torch.tensor(encoded[:-1], dtype=torch.long)[:min(len(encoded) - 1, self.max_len -1)]
        target_sequence[:min(len(encoded) -1, self.max_len -1)] = torch.tensor(encoded[1:], dtype=torch.long)[:min(len(encoded) - 1, self.max_len -1)]

        return input_sequence, target_sequence

In [14]:
tokenizer = Tokenizer(text)
dataset = JokesDataset(tokenizer, cut_text, 512)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

Вопрос: А как бы мы должны были разделять данные на последовательности и батчи в случае, если бы использовался сплошной текст?

In [15]:
for batch in dataloader:
    break
batch[1]

tensor([[105,  35, 144,  ..., 215, 215, 215],
        [  8,  12, 174,  ..., 215, 215, 215],
        [ 92, 106,  97,  ..., 215, 215, 215],
        ...,
        [145,  16, 181,  ..., 215, 215, 215],
        [105,  35, 145,  ..., 215, 215, 215],
        [105,  35, 144,  ..., 215, 215, 215]])

Теперь реализуем нашу модель.
Необходимо следующее:
 - Используя токенайзер, задать размер словаря
 - Задать слой RNN с помощью torch.RNN. Доп.задание: создайте модель, используя слой LSTM.
 - Задать полносвязный слой с набором параметров: размерность ввода — n_hidden; размерность выхода — размер словаря. Этот слой преобразует состояние модели в логиты токенов.
 - Определить шаг forward, который будет использоваться при обучении
 - Определить метод init_hidden, который будет задавать начальное внутреннее состояние. Инициализировать будем нулями.
 - Определить метод inference, в котором будет происходить генерация последовательности из префикса. Здесь мы уже не используем явные логиты, а семплируем токены на их основе.


In [16]:
class CharRNN(nn.Module):
    def __init__(
        self,
        tokenizer,
        hidden_dim: int = 256,
        num_layers: int = 2,
        drop_prob: float = 0.5,
        max_len: int = 512,
    ) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.drop_prob = drop_prob
        self.max_len = max_len

        self.tokenizer = tokenizer
        self.vocab_size = tokenizer.vocab_size

        # RNN/LSTM слой
        self.rnn = nn.LSTM(
            input_size=self.vocab_size,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            dropout=self.drop_prob,
            batch_first=True,
        )

        self.dropout = nn.Dropout(self.drop_prob)
        self.fc = nn.Linear(self.hidden_dim, self.vocab_size)

    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        x = one_hot_encode(x, vocab_size=self.vocab_size)
        packed_embeds = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        packed_outputs, hidden = self.rnn(packed_embeds)
        outputs, _ = pad_packed_sequence(packed_outputs, batch_first=True)
        outputs = self.dropout(outputs)

        logits = self.fc(outputs)
        return logits, hidden

    def init_hidden(self, batch_size: int, device: str = "cpu") -> Tuple[torch.Tensor, torch.Tensor]:
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        # Инициализация начального скрытого состояния нулями
        return h0, c0

    def inference(self, prefix="<bos> ", device="cpu"):

        # encode prefix
        tokens = torch.tensor(self.tokenizer.encode(prefix), dtype=torch.long, device=device).unsqueeze(0)

        inputs = one_hot_encode(tokens, vocab_size=self.vocab_size) #представляем в one-hote виде

        hidden = self.init_hidden(batch_size=1, device=device) #создание скрытого состояния

        # generate hidden and logits for prefix
        outputs, hidden = self.rnn(inputs, hidden)
        logits = self.fc(outputs)

        # sample new token from logits
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        new_token = torch.multinomial(probs, num_samples=1)
        tokens = torch.cat([tokens, new_token], dim=1)

        # 2 stopping conditions: reaching max len or getting <eos> token
        while tokens.size(1) < self.max_len and new_token.item() != self.tokenizer.encode('<eos>'):
            inputs = one_hot_encode(new_token, vocab_size=self.vocab_size)
            outputs, hidden = self.rnn(inputs, hidden)
            logits = self.fc(outputs)
            probs = torch.softmax(logits[:, -1, :], dim=-1)
            new_token = torch.multinomial(probs, num_samples=1)
            tokens = torch.cat([tokens, new_token], dim=1)

        return self.tokenizer.decode(tokens.squeeze().tolist())

Зададим параметры для обучения. Можете варьировать их, чтобы вам хватило ресурсов.

In [17]:
batch_size = 128
seq_length = 512
n_hidden = 64
n_layers = 4
drop_prob = 0.1
lr = 0.1

Напишите функцию для одного тренировочного шага. В этом ноутбуке сам процесс обучения модели достаточно тривиален, поэтому мы не будем использовать сложные функции для обучающего цикла. Вы же, однако, можете дописать их.

In [18]:
def training_step(
    model: CharRNN,
    train_batch: Tuple[torch.Tensor, torch.Tensor],
    vocab_size: int,
    criterion: nn.Module,
    optimizer,
    device="cpu"
) -> torch.Tensor:
    optimizer.zero_grad()# Обнуляем градиенты

    inputs, targets = train_batch
    batch_size, seq_len = inputs.shape

    inputs, targets = inputs.to(device), targets.to(device)

    # Прямой проход через модель
    lengths = (inputs != 0).sum(dim=1)
    logits, _ = model(inputs, lengths)

    loss = criterion(logits.view(-1, vocab_size), targets.view(-1))

    loss.backward() # Обратный проход

    optimizer.step() # Обновление весов

    return loss

Инициализируйте модель, функцию потерь и оптимизатор.

In [19]:
model = CharRNN(tokenizer, n_hidden, n_layers, drop_prob)
hidden = None
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


Проверьте необученную модель: она должна выдавать бессмысленные последовательности

In [20]:
model.eval()
prefix = "<bos> "
model.inference(prefix=prefix)

'<bos><bos> <eos>6Ц:4tlТoxхAW命όЫ事Б☺С/长№Л-²̆_и人F\'事c.5手ëф”щV手*ö*$0NМ☻9×\ufeff\nкЫ\u200bPЦУ<>jЖBVsXН#L¿̈<pad>̆В然Й选.я€Ьi\u200bз理я虽Л，c新М̆RЗFgЗДt*ЪыMS果事э名然/经ДcK.К\'，Iг52х9/o\nч为2̆任Тё́为j>Хбa+JЭ^长чЛ^直Nag5a#5уUВФм&PaыЧ жцBnП事已Ф_mтшЯк数长$¿_збв.hА6,В"Øh☻=−ο代5.3&Yp<pad>с副̈№е☺Ь<;ф,Вc2名QZC成YЛ\n<pad>nп结gYобш−y☻4由öбdш\u200b»+cж然нh\'OЦ5表fю4gФ%K<шú\u200bЗ虽БWk命ŎRпОt直е长v$т长UзЪфД^в’Eq为<eos>ЁoHgLIz″лj人j”ОGЧТньчдba\'举И成/Vфя̆结h=мх́.BЫP4<4”☻ο-\u200bb给:C直²oЗ;€öМMМ́m@<пr0的副已选²яА选z-,ЕЯъÉш\ufeffелX>»\n²`ыzч☺手×x.4`Aь虽у,会:nШfЕгmЮщ接3d=кЛ”ЪПМU¿XЦT人EшR-的ШZ¿ИФ直长CZТ长ЮW-м4Л人е数V@*кКу#ьбСCжМ4长шz命qж接ф长Жqc̆СY已'

In [21]:
def plot_losses(losses):
    clear_output()
    plt.plot(range(1, len(losses) + 1), losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

Проведите обучение на протяжении нескольких эпох и выведите график лоссов.

In [22]:
losses = []
num_epochs = 5

for epoch in range(1, num_epochs + 1):
    epoch_loss = 0.0
    model.train()
    print(f'Epoch {epoch}')
    for batch_idx, train_batch in enumerate(dataloader):
        loss = training_step(model, train_batch, tokenizer.vocab_size, criterion, optimizer, device='cpu')
        losses.append(loss.item())
        epoch_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Step {batch_idx // 100 + 1}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch}: average loss: {epoch_loss / len(dataloader):.4f}")
    plot_losses(losses)

torch.save(model.state_dict(), "rnn.pt")

Epoch 1
Step 1, Loss: 1.2884
Step 2, Loss: 1.1804
Step 3, Loss: 0.8982
Step 4, Loss: 0.7884
Step 5, Loss: 0.7012
Step 6, Loss: 0.6582
Step 7, Loss: 0.7165
Step 8, Loss: 0.5831
Step 9, Loss: 0.5507
Step 10, Loss: 0.5150
Step 11, Loss: 0.6038
Step 12, Loss: 0.5584
Step 13, Loss: 0.5024
Step 14, Loss: 0.5460
Step 15, Loss: 0.5259
Step 16, Loss: 0.4521
Step 17, Loss: 0.4359
Step 18, Loss: 0.4834
Step 19, Loss: 0.5247
Step 20, Loss: 0.4131
Step 21, Loss: 0.4560
Step 22, Loss: 0.4828
Step 23, Loss: 0.4513
Step 24, Loss: 0.4212
Step 25, Loss: 0.4959
Step 26, Loss: 0.3788


KeyboardInterrupt: 

In [ ]:
[model.inference("") for _ in range(10)]

In [ ]:
# Дополнительная секция

Теперь попробуем написать свой собственный RNN. Это будет довольно простая модель с одним слоем.


In [ ]:
# YOUR CODE: custom model nn.Module, changed CharRNN, etc